In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys,os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

sys.path.append("../likelihood/")

import PSR_counts as gc
from likelihood import ll

# 1. The expected number of PS counts

Here, we describe how we calculate the expected number of PS counts, given the model parameters.  These computations are implemented in the file `get_counts_inline.pyx` in the `likelihood/` subfolder.  First, we describe the computation for a general PS population, with density function $\rho(s,\ell,b)$, where $(s,\ell,b)$ are spatial coordinates with origin at the Earth, and luminosity function $dN/dL$.  Note that $\ell$ and $b$ are Galactic longitude and latitude, respectively, while $s$ is the distance from the Earth.  The expected number of counts in a given spatial bin (labeled by $i,j$) and flux bin (labelled by $k$) is given by the integral

$$
N_{i,j,k} = \omega_{i,j,k} \int_{\Delta \Omega_{i,j}} d\ell d b \rho(s,\ell,b) s^2 \int_{4 \pi s^2 S_k^\text{min}}^{4 \pi s^2 S_k^\text{max}} {dN \over dL} dL \,,
$$ 

where $\omega_{i,j,k}$ is the efficiency factor for pulsar-like PS detection in that bin.  Note that we have assumed that $\omega_{i,j,k}$ is constant within the bin, so that we may bring it outside of the integral.
 

In the code, for both the disk and the Bulge, we normalize both $\rho$ and $dN/dL$ such that the full integral over all of space and flux is equal to the total number of sources $N$ for that population.  The number of sources is one of the model parameters.  To work with this convention, we must know how to properly normalize $\rho$ and $dN/dL$ .

The normalization of $dN/dL$ is common to both the disk and Bulge.  Let 

$$
{dN \over dL} = {N_0 \over L^\beta}
$$

for $L$ in the range $[L_\text{min}, L_\text{max}]$ and be zero outside of this range.  We will normalize $dN/dL$ so that it integrates to unity, which implies that
 
$$
N_0 = {\beta - 1 \over L_\text{min}^{1-\beta} - L_\text{max}^{1 - \beta} } \,.
$$  


We discuss $\rho$ now for the disk and Bulge.  We will use the normalization that $\int d^3x \rho = N$.

Below, we take a few artibtrary choices to illustrate to functionality.

In [2]:
N_stars = 10.0 #example, why not
omega_ijk = 1.0 #example, why not
i=3 #example, why not
j=3 #example, why not
k=1 #example, why not

Ns = 100 #number of s bins in s integration
Nang = 2 #number of angular bins in angular integration


#These are also adjustable 
Lmin=1.0e31
Lmax = 1.0e36

#Theta mask: this masks the inner \theta degrees when calculating the expected number of counts
theta_mask = 2.0 #degrees

## 1.1. Bulge sources 

We characterize the Bulge PS population by

$$
\rho_\text{bulge} = {C \over r^\alpha} \,,
$$

where $r$ is the distance from the Galactic Center, so long as $r < r_\text{cut}$ .  $\rho$ is equal to zero for larger values of $r$ .  To have to proper normalization, it is straightforward to verify that
 
$$
C = {(3 - \alpha) N_\text{bulge} \over 4 \pi r_\text{cut}^{3 - \alpha} } \,.
$$

In [4]:
alpha=2.60
beta=1.20
rcut = 3.0 #kpc

N_bulge = gc.Nbulge_full_ang_ijk(i,j,k,N_stars,omega_ijk,alpha,beta,rcut,Lmin,Lmax,Ns,Nang,theta_mask) 
print N_bulge

n = 2.35
z0 = 0.7 #kpc
sigma = 1.528 #kpc
beta=1.20

smax = 40.0 #kpc , how far out to integrate to?

N_disk = gc.Ndisk_full_ang_ijk(i,j,k,N_stars,omega_ijk,n,sigma,z0,beta,Lmin,Lmax,Ns,Nang,smax,theta_mask) 
print N_disk

## 1.2. Disk sources

The disk is modeled by the distribution 

$$
\rho_\text{disk} = C R^n e^{-R / \sigma} e^{-|z| / z_0} \,,
$$

and we find that in this case

$$
C = {N_\text{disk} \over 4 \pi z_0 \sigma^{n+2} \Gamma(n+2)} \,.
$$

In [ ]:
n = 2.35
z0 = 0.7 #kpc
sigma = 1.528 #kpc
beta=1.20

smax = 40.0 #kpc , how far out to integrate to?

N_disk = gc.Ndisk_full_ang_ijk(i,j,k,N_stars,omega_ijk,n,sigma,z0,beta,Lmin,Lmax,Ns,Nang,smax,theta_mask) 
print N_disk

Now, let's calculate the total number of total bulge and disk counts, for the prior.
Here, we integrate over fluxes from $1.8 \times 10^{-5}$ MeV cm$^{-2}$ s$^{-1}$ to $6.539 \times 10^{-4}$ MeV cm$^{-2}$ s$^{-1}$.

In [6]:
Nang_total = 100 #We use a larger number of angular bins here, since we are integrating over the full sky
N_bulge_total = gc.Nbulge_total(N_stars,alpha,beta,rcut,Lmin,Lmax,Ns,Nang_total,theta_mask)
N_disk_total = gc.Ndisk_total(N_stars,n,sigma,z0,beta,Lmin,Lmax,Ns,Nang_total,smax,theta_mask)
print N_bulge_total,N_disk_total

# 2. The likelihood function

The `GCE-2FIG` code package uses the likelihood function presented in [1705.00009](https://arxiv.org/pdf/1705.00009.pdf).  The likelihood is implemented in the file `likelihood.pyx` in the `likelihood/` subfolder.  The sky is spatially binned in a cartesian grid, and the data consists of the number of pulsar candidates detected in each bin.  The model parameters characterize the disk and Bulge point source populations, and as we scan over the model parameters we calculated the expected number of detected sources in each bin, utilizing the _Fermi_-provided efficiency function for detecting sources at a given flux and spatial position.  The likelihood is then given by the product over all pixels of the Poisson probabilities to observe the data counts given the predicted model counts.

As an option, we also incorporate the prior distribution in [1705.00009](https://arxiv.org/pdf/1705.00009.pdf).

# Appendix: code implementation

Here, we describe in more detail how we implement the computations of the expected number of PS counts in the file `get_counts_inline.pyx`.

The cartesian grid is given by 12 linear-spaced bins in the inner 40$^\circ$ of the Milky Way:

```c
cdef double[::1] ang_boundaries = np.linspace(-20.,20.,13,dtype=DTYPE)
``` 

The flux-bin boundaries are given by

```python
fluxvals = [1.00000000e-06, 1.46779927e-06, 2.15443469e-06, 3.16227766e-06, 4.64158883e-06, 6.81292069e-06, 1.00000000e-05, 3.16227766e-05, 1.00000000e-04]
```

in units of MeV/cm^2/s.

The function 

```c
double Nbulge_full_ang_ijk(int i, int j, int k, double Nbulge, double omega_ijk, double alpha, double beta,double rcut , double Lmin, double Lmax ,int Ns ,int Nang, double theta_mask )
```

return the number of counts in the Bulge in spatial $\ell$ bin `i`, $b$ bin `j`, and flux bin `k`.  Most of the parameters above are defined previously.  However, there are a few parameters that are specific to the calculation method.  These include

    1. int Ns: the number of points in the numerical integral over s
    2. int Nang: the number of latitude and longitude points in the numerical integral over the pixel
    3. double theta_mask: the radius in degrees fro the Galactic Center that will be masked when computing the number of counts
  
The code itself proceeds by first initializing relevant quantities, such as

```python
cdef double ell_start = ang_boundaries[i]
cdef double b_start = ang_boundaries[j]
cdef double d_ang = (ang_boundaries[1] - ang_boundaries[0])/float(Nang)
```

and  calculating relevant pre-factors, such as 

```python
pref_rho = omega_ijk * Nbulge * (3. - alpha) / 4. / pi / pow(rcut,3 - alpha)
```

and 

```python
pref_L_norm = 1./(pow(Lmin,1-beta) - pow(Lmax,1-beta))
```

Then, the bulk of the code proceeds through the loops used to perform the angular integration and the integration over $s$:

```python
for i_b in range(Nang):
    b = b_start + i_b*d_ang + d_ang/2.
    for i_ell in range(Nang):
        ell = ell_start + i_ell*d_ang + d_ang/2.
        coslval = cos(ell * degtorad)
        cosbval = cos(b * degtorad)

        if cosbval * coslval < costhetaval: #is it masked?

            incl = 1. - (1. - pow(rcut/rodot,2) ) * pow(coslval*cosbval,2)
            if incl > 0: # should we bother, or is the answer going to be zero?
                integral = 0.0
                s = smin
                for l in range(Ns): #Loop over `s` for `s` integral

                    flux_min_eval = max(Lmin/s**2,4*pi*fluxmin)
                    flux_max_eval = min(Lmax/s**2,4*pi*fluxmax)

                    if flux_max_eval < flux_min_eval:
                        pref_L = 0.0
                    else:
                        pref_L = (pow(flux_min_eval, 1.-beta) - pow(flux_max_eval, 1.-beta))*pref_L_norm

                    r_squared = (s*cosbval*coslval - rodot)**2 + s**2*(1. - pow(cosbval*coslval,2) )
                    if r_squared < pow(rcut,2):
                        integral += pref_L * pow(s,4.-2.*beta)*pow(r_squared,-alpha/2.)
                    s += ds

                total_res += cosbval * integral 
```

Above, `i_ell` is the index over longitude, `i_b` is the index of latitude, and `l` is the index over the $s$ bins.  In the first few lines we simply update the new values for `b` and `ell` and their cosines.  We then see if these values of `b` and `ell` are masked.  The statement `if incl > 0:` looks to see if we are too far away from the GC, such that we will completely miss the bulge.  This is relevant given the finite extent of the bulge, but this step is not needed for the disk contribution.  Then, we begin looping over the `s` integral, through the index `l`.  First, we find the true limits of the flux integral, given that fact that the luminosity function is cut off at `Lmin` and `Lmax`.  Then, we perform the integral over `L` to get `pref_L`.  The quantity `integral` contains both the integral over `s` and the integral over `L`.  

Finally, we return 

```python
return total_res * d_ang**2. * degtorad**2 * ds * pref_rho
```  

Note that the disk proceeds similarly, as do the functions that compute the full-sky integrals for the prior distribution.